In [34]:
import config as cfg
import os
import pandas as pd
import networkx as nx
import geopandas as gpd
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import plotly.graph_objects as go

# Load data
all_viajes = pd.read_csv('/Users/caro/Desktop/thesis_project/mobility_data/VIAJES/all_viajes_week_0222.csv')
gdf = gpd.read_file(cfg.ZONIFICACION_DATA / 'distritos/madrid_gdf.geojson')  # Load your GeoJSON file into a GeoDataFrame
gdf = gdf.to_crs(epsg=3042)

# Filter data
filtered_df = all_viajes.loc[(all_viajes['actividad_origen'] == 'casa')]

In [53]:
# Function to generate node positions based on GeoDataFrame
def get_positions(gdf):
    return {
        int(row['ID']): (row['geometry'].centroid.x, row['geometry'].centroid.y)
        for idx, row in gdf.iterrows()
    }

# Define the graph based on DataFrame
def define_graph(df, weight_column):
    G = nx.DiGraph()
    for idx, row in df.iterrows():
        G.add_edge(row['origen'], row['destino'], weight=row[weight_column], type=row['renta'])
    return G

# Set edge attributes like color and width for visualization
def set_art(G, weight_scale=60):
    edge_colors = []
    edge_widths = []
    for u, v, data in G.edges(data=True):
        edge_colors.append('blue' if data['type'] == '>15' else 'red')
        edge_widths.append(max(0.5, data['weight'] / weight_scale))
    return edge_colors, edge_widths

# Convert graph to Plotly format
def plotly_graph(G, positions, edge_colors, edge_widths, node_size=20):
    # Extract node positions
    node_x = [positions[node][0] for node in G.nodes()]
    node_y = [positions[node][1] for node in G.nodes()]

    # Create node scatter trace
    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        marker=dict(size=node_size, color='white', line=dict(width=2, color='#888')),
        text=[f'Node {node}' for node in G.nodes()],
        hoverinfo='text'
    )

    # Prepare edges in batches based on edge color (blue, red)
    edge_traces = []
    for edge_color in ['blue', 'red']:
        edge_x, edge_y = [], []
        for i, (u, v, data) in enumerate(G.edges(data=True)):
            if edge_colors[i] == edge_color:
                x0, y0 = positions[u]
                x1, y1 = positions[v]
                edge_x.extend([x0, x1, None])
                edge_y.extend([y0, y1, None])

        # Create edge trace for each color
        edge_traces.append(
            go.Scatter(
                x=edge_x, y=edge_y,
                line=dict(width=edge_widths[i], color=edge_color),
                hoverinfo='none',
                mode='lines'
            )
        )

    # Create a layout for the graph
    layout = go.Layout(
        showlegend=True,
        hovermode='closest',
        margin=dict(b=0, l=0, r=0, t=0),
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
    )

    # Combine edge and node traces
    fig = go.Figure(data=edge_traces + [node_trace], layout=layout)
    fig.show()

# ANALYSIS ---------------------------------------------------------------------------------------------------------

def check_in_weights(G):
    in_weights = {}
    for node in G.nodes():
        total_in_weight = sum(data['weight'] for u, v, data in G.in_edges(node, data=True))
        in_weights[node] = total_in_weight
        print(f"Node {node} Total In-weight: {total_in_weight}")
    return in_weights

def check_out_weights(G):
    out_weights = {}
    for node in G.nodes():
        total_out_weight = sum(data['weight'] for u, v, data in G.out_edges(node, data=True))
        out_weights[node] = total_out_weight
        print(f"Node {node} Total Out-weight: {total_out_weight}")
    return out_weights

def print_node_degrees(G):
    print("Node Degrees (In-degree, Out-degree, Total degree):")
    for node in G.nodes():
        in_degree = G.in_degree(node)
        out_degree = G.out_degree(node)
        total_degree = G.degree(node)  # Total degree (in + out)
        print(f"Node {node}: In-degree = {in_degree}, Out-degree = {out_degree}, Total degree = {total_degree}")



In [54]:
G = define_graph(filtered_df, 'viajes')
positions = get_positions(gdf)
edge_colors, edge_widths = set_art(G)
plotly_graph(G, positions, edge_colors, edge_widths)

In [51]:
filtered_df.head()

,fecha,periodo,origen,destino,distancia,actividad_origen,actividad_destino,estudio_origen_posible,estudio_destino_posible,residencia,renta,edad,sexo,viajes,viajes_km
0,20220207,0,2807901,2807901,0.5-2,casa,frecuente,no,no,28,>15,0-25,hombre,20.700,14.144
1,20220207,0,2807901,2807901,0.5-2,casa,frecuente,no,no,28,>15,0-25,mujer,25.476,15.796
2,20220207,0,2807901,2807901,0.5-2,casa,frecuente,no,no,28,>15,25-45,hombre,102.263,69.730
3,20220207,0,2807901,2807901,0.5-2,casa,frecuente,no,no,28,>15,25-45,mujer,86.178,63.775
4,20220207,0,2807901,2807901,0.5-2,casa,frecuente,no,no,28,>15,45-65,hombre,43.469,31.687


In [37]:
print_node_degrees(G)

Node Degrees (In-degree, Out-degree, Total degree):
Node 2807901: In-degree = 21, Out-degree = 21, Total degree = 42
Node 2807902: In-degree = 21, Out-degree = 21, Total degree = 42
Node 2807903: In-degree = 21, Out-degree = 21, Total degree = 42
Node 2807904: In-degree = 21, Out-degree = 21, Total degree = 42
Node 2807905: In-degree = 21, Out-degree = 21, Total degree = 42
Node 2807906: In-degree = 21, Out-degree = 21, Total degree = 42
Node 2807907: In-degree = 21, Out-degree = 21, Total degree = 42
Node 2807908: In-degree = 21, Out-degree = 21, Total degree = 42
Node 2807909: In-degree = 21, Out-degree = 21, Total degree = 42
Node 2807910: In-degree = 21, Out-degree = 21, Total degree = 42
Node 2807911: In-degree = 21, Out-degree = 21, Total degree = 42
Node 2807912: In-degree = 21, Out-degree = 21, Total degree = 42
Node 2807913: In-degree = 21, Out-degree = 21, Total degree = 42
Node 2807914: In-degree = 21, Out-degree = 21, Total degree = 42
Node 2807915: In-degree = 21, Out-degr

In [38]:
# Check total in-weights for each node
in_weights = check_in_weights(G)


Node 2807901 Total In-weight: 110.06
Node 2807902 Total In-weight: 82.30200000000002
Node 2807903 Total In-weight: 72.84800000000001
Node 2807904 Total In-weight: 92.35799999999999
Node 2807905 Total In-weight: 78.93900000000001
Node 2807906 Total In-weight: 72.571
Node 2807907 Total In-weight: 117.01300000000003
Node 2807908 Total In-weight: 89.65700000000001
Node 2807909 Total In-weight: 85.24400000000001
Node 2807910 Total In-weight: 79.20400000000001
Node 2807911 Total In-weight: 96.47600000000001
Node 2807912 Total In-weight: 80.22000000000001
Node 2807913 Total In-weight: 81.09199999999998
Node 2807914 Total In-weight: 76.07000000000001
Node 2807915 Total In-weight: 88.11600000000001
Node 2807916 Total In-weight: 76.476
Node 2807917 Total In-weight: 79.85300000000001
Node 2807918 Total In-weight: 78.66600000000001
Node 2807919 Total In-weight: 85.483
Node 2807920 Total In-weight: 88.811
Node 2807921 Total In-weight: 84.11000000000001


In [39]:
# Check total out-weights for each node
out_weights = check_out_weights(G)


Node 2807901 Total Out-weight: 85.75099999999999
Node 2807902 Total Out-weight: 105.585
Node 2807903 Total Out-weight: 96.69999999999999
Node 2807904 Total Out-weight: 93.526
Node 2807905 Total Out-weight: 103.44
Node 2807906 Total Out-weight: 73.219
Node 2807907 Total Out-weight: 91.08600000000001
Node 2807908 Total Out-weight: 77.297
Node 2807909 Total Out-weight: 76.337
Node 2807910 Total Out-weight: 85.81899999999999
Node 2807911 Total Out-weight: 79.54500000000002
Node 2807912 Total Out-weight: 101.81499999999998
Node 2807913 Total Out-weight: 72.22800000000001
Node 2807914 Total Out-weight: 84.46
Node 2807915 Total Out-weight: 93.66
Node 2807916 Total Out-weight: 101.79
Node 2807917 Total Out-weight: 73.61000000000001
Node 2807918 Total Out-weight: 85.867
Node 2807919 Total Out-weight: 68.965
Node 2807920 Total Out-weight: 76.384
Node 2807921 Total Out-weight: 68.485


In [48]:
nx.community.louvain_communities(G)

[{2807904, 2807905, 2807916},
 {2807901, 2807902, 2807907, 2807912, 2807921},
 {2807906, 2807910, 2807911, 2807914, 2807917, 2807919},
 {2807903, 2807908, 2807909, 2807913, 2807915, 2807918, 2807920}]